# Part 1: Python – Data Cleaning & Prep (~1 Hour)

## 06/17/2025

You are given this messy CSV:

delivery_id,driver,warehouse,retail_location,timestamp,product,qty,status
1001,Sam,W1,Store-A,"2024-06-01 09:00",Beer Keg,5,Delivered
1002,Lisa,W1,Store-B,"06/01/2024 10:15",Soda Crate,7,Delivered
1003,Sam,W2,Store-C,2024/06/01 12:00,Water Jug,2,damaged
1004,Lisa,W1,Store-D,INVALID_TIME,Beer Keg,5,Delivered
1005,,W1,Store-A,"2024-06-01 13:00",Soda Crate,8,Delivered
1006,John,W3,Store-B,"2024-06-01 14:30",,3,Delivered
1007,Lisa,W1,Store-A,"2024-06-01 15:00",Beer Keg,,Delivered

## 📋 Tasks:
Clean the data:

Remove rows with missing driver, timestamp, or product

Normalize the timestamp to datetime (all formats to %Y-%m-%d %H:%M)

Set qty to 0 if missing or non-numeric

Add fields:

A new delivery_hour column (e.g. 9, 10, 11…)

A success_flag column: 1 if status = 'Delivered', else 0

Save cleaned dataset to either CSV or in-memory SQLite for SQL use in part 2.

## ✅ Set Up The Workspace

In [1]:
from io import StringIO
import pandas as pd
import sqlite3

## ✅ Simulate the Raw CSV as a String

In [2]:
raw_csv = """
delivery_id,driver,warehouse,retail_location,timestamp,product,qty,status
1001,Sam,W1,Store-A,"2024-06-01 09:00",Beer Keg,5,Delivered
1002,Lisa,W1,Store-B,"06/01/2024 10:15",Soda Crate,7,Delivered
1003,Sam,W2,Store-C,2024/06/01 12:00,Water Jug,2,damaged
1004,Lisa,W1,Store-D,INVALID_TIME,Beer Keg,5,Delivered
1005,,W1,Store-A,"2024-06-01 13:00",Soda Crate,8,Delivered
1006,John,W3,Store-B,"2024-06-01 14:30",,3,Delivered
1007,Lisa,W1,Store-A,"2024-06-01 15:00",Beer Keg,,Delivered
"""

df = pd.read_csv(StringIO(raw_csv))

✅ That reads the CSV into a DataFrame, like an Excel table.

## ✅ Step 2: Clean the Data
We’ll clean in layers:

### 🔹 Remove rows with missing driver, timestamp, or product:

In [3]:
df_clean = df.dropna(subset=['driver', 'timestamp', 'product']).copy()

### 🔹 Convert timestamp to datetime (and handle errors):

In [ ]:
# Checking for date formats
def try_parse_date(val):
    for fmt in ("%Y-%m-%d %H:%M", "%m/%d/%Y %H:%M", "%Y/%m/%d %H:%M"):
        try:
            return pd.to_datetime(val, format=fmt)
        except:
            continue
    return pd.NaT  # fallback if all formats fail

df_clean['timestamp'] = df_clean['timestamp'].apply(try_parse_date)
df_clean = df_clean.dropna(subset=['timestamp']) # drop rows where timestamp failed

### 🔹 Fix qty — if missing or not numeric, set to 0:

In [5]:
df_clean['qty'] = pd.to_numeric(df_clean['qty'], errors='coerce').fillna(0).astype(int)

## ✅ Step 3: Add New Columns

### 🔹 Add delivery_hour (hour from timestamp):

In [6]:
df_clean['delivery_hour'] = df_clean['timestamp'].dt.hour

### 🔹 Add success_flag (1 if status is 'Delivered'):

In [7]:
df_clean['success_flag'] = (df_clean['status'].str.lower() == 'delivered').astype(int)

## ✅ Step 4: Save for SQL or Output

### To CSV:

In [8]:
df_clean.to_csv('cleaned_deliveries.csv', index=False)

### Or for SQL use (in-memory SQLite):

In [ ]:
conn = sqlite3.connect(':memory:')
df_clean.to_sql('deliveries', conn, index=False, if_exists='replace')

# Part 2: SQL — Delivery Metrics from Cleaned Data

Goal: You’ll now query the cleaned delivery data to generate operational insights.

We’re assuming your cleaned data from Python is in a table called deliveries.

## 📦 Table: deliveries

| Column           | Type     | Example          |
| ---------------- | -------- | ---------------- |
| delivery\_id     | int      | 1001             |
| driver           | string   | "Sam"            |
| warehouse        | string   | "W1"             |
| retail\_location | string   | "Store-A"        |
| timestamp        | datetime | 2024-06-01 09:00 |
| product          | string   | "Beer Keg"       |
| qty              | int      | 5                |
| status           | string   | "Delivered"      |
| delivery\_hour   | int      | 9                |
| success\_flag    | int      | 1                |

## 🎯 Tasks

### 🧮 1. Total Quantity Delivered Per Product

Expected Output:
product     | total_delivered
------------|----------------
Beer Keg    | 10
Soda Crate  | 7
Water Jug   | 2

Write a query that:
- Groups by `product`
- Sums the `qty` field
- Only includes rows where `success_flag = 1`

### Solution:

In [ ]:
SELECT
    product,
    sum(qty) as total_delivered
FROM
    scratchpad.deliveries
WHERE
    success_flag = 1
GROUP BY
    product

### 🕒 2. Peak Delivery Hours

Expected Output:
delivery_hour | total_deliveries
--------------|------------------
9             | 1
13            | 1
15            | 1

Write a query that:
- Groups by `delivery_hour`
- Counts total deliveries
- Filters to only successful ones
- Orders by delivery_hour

### Solution:

In [ ]:
SELECT
    delivery_hour,
    count(case when status = 'Delivered' then 1 end) as total_deliveries
FROM
    scratchpad.deliveries
WHERE
    success_flag = 1
GROUP BY
    delivery_hour
ORDER BY
    delivery_hour

### 👷‍♂️ 3. Total Deliveries Per Driver

Expected Output:
driver | deliveries
---|---
Sam    | 1
Lisa   | 1

Write a query that:
- Counts total successful deliveries
- Groups by `driver`
- Orders by deliveries DESC

### Solution:

In [ ]:
SELECT
    driver,
    count(case when status = 'Delivered' then 1 end) as deliveries
FROM
    scratchpad.deliveries
WHERE
    success_flag = 1
GROUP BY
    driver
ORDER BY
    deliveries DESC

### Bonus Challenge (Optional)

Which warehouse has the lowest delivery success rate?

You’ll need to:

🔹 Count total deliveries per warehouse

🔹 Count successful ones

🔹 Calculate a success % per warehouse

🔹 Order by that % ascending

### Solution:

In [ ]:
SELECT
    warehouse,
    sum(success_flag),
    ROUND(100.0 * SUM(success_flag) / COUNT(*), 2) as success_pct
FROM
    scratchpad.deliveries
GROUP BY
    warehouse
ORDER BY
    success_pct